# Limit Spread vs Fill

Analyze the relationship between limit-order distance from market at submit time and fill outcomes.

Source: `{TEAM_ID}/order_events.csv` with:
- `limit_price`
- `market_price_at_submit`
- `fill_quantity`
- `quantity`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook
from config import TEAM_ID

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None

## Data Loading — Limit Order Events

Loads the order events log and filters it to limit-order events only, then parses the tier tag from the order annotation string. The head preview confirms that `limit_price`, `market_price_at_submit`, and `fill_quantity` columns are present — the three values required to compute limit spread and fill outcome. All subsequent analysis is restricted to limit orders since market orders have no spread by definition.

## Per-Order Spread Computation

This cell aggregates the event log to one row per limit order and computes the favorable spread — the distance between the submitted limit price and the market price at submission, expressed in basis points in the direction that makes the limit passive. The `analysis_df` DataFrame produced here is the primary input for all spread-versus-fill charts. A head preview confirms spread data is populated before any plots are generated.

## Spread vs Fill Rate — 4-Panel Chart and Summary Tables

These four panels and two tables investigate how limit spread distance affects fill outcomes. Top-left shows the distribution of spreads at submission; top-right scatter plots fill percentage against spread colored by tier; bottom-left shows the fill-rate curve as spread widens across bucketed ranges; and bottom-right shows average fill percentage by tier, directly testing whether strong-tier near-market limits fill better than wide weak-tier limits. The summary tables provide numerical precision to the patterns seen in the plots.

In [ ]:
import re

df_events = read_csv_from_store(f'{TEAM_ID}/order_events.csv')
if df_events is None:
    raise ValueError('order_events.csv is required. Run a backtest with order-event logging enabled.')

required_cols = {
    'order_id', 'date', 'status', 'direction', 'quantity', 'fill_quantity',
    'order_type', 'limit_price', 'market_price_at_submit', 'tag'
}
missing = sorted(required_cols - set(df_events.columns))
if missing:
    raise ValueError(
        'Missing required columns in order_events.csv: '
        f"{missing}. Re-run backtest after latest logging changes."
    )

df_events['date'] = pd.to_datetime(df_events['date'])
for col in ['quantity', 'fill_quantity', 'fill_price', 'limit_price', 'market_price_at_submit']:
    if col in df_events.columns:
        df_events[col] = pd.to_numeric(df_events[col], errors='coerce')


def parse_tag_value(tag, key):
    if pd.isna(tag):
        return np.nan
    m = re.search(rf'{key}=([^;]+)', str(tag))
    return m.group(1) if m else np.nan


df_events['tier'] = df_events['tag'].apply(lambda t: parse_tag_value(t, 'tier')).fillna('unknown')

# Keep limit-order lifecycle rows only
is_limit = df_events['order_type'].astype(str).str.contains('Limit', case=False, na=False)
df_limit = df_events[is_limit].copy()
if df_limit.empty:
    raise ValueError('No limit-order events found in order_events.csv.')

print(f'limit-order events: {len(df_limit):,}')
display(df_limit.head())

In [ ]:
# Aggregate each limit order into one row
base = (
    df_limit.sort_values('date')
    .groupby('order_id', as_index=False)
    .agg(
        symbol=('symbol', 'first'),
        tier=('tier', 'first'),
        direction=('direction', 'first'),
        order_type=('order_type', 'first'),
        quantity=('quantity', 'first'),
        limit_price=('limit_price', 'first'),
        market_price_at_submit=('market_price_at_submit', 'first'),
        submitted_at=('date', 'min'),
        final_at=('date', 'max')
    )
)

fills = (
    df_limit.groupby('order_id', as_index=False)['fill_quantity']
           .sum()
           .rename(columns={'fill_quantity': 'filled_qty'})
)

final_status = (
    df_limit.sort_values('date')
            .groupby('order_id')
            .tail(1)[['order_id', 'status']]
            .rename(columns={'status': 'final_status'})
)

order_summary = base.merge(fills, on='order_id', how='left').merge(final_status, on='order_id', how='left')

order_summary['abs_qty'] = order_summary['quantity'].abs().replace(0, np.nan)
order_summary['fill_pct'] = (
    order_summary['filled_qty'].abs() / order_summary['abs_qty']
).fillna(0.0).clip(0.0, 1.0)
order_summary['is_full_fill'] = order_summary['fill_pct'] >= 0.999

# Signed spread: positive means limit above market
order_summary['signed_spread_pct'] = (
    (order_summary['limit_price'] - order_summary['market_price_at_submit'])
    / order_summary['market_price_at_submit']
)

# Direction-adjusted distance from market (always >=0 for passive limit pricing)
dir_lower = order_summary['direction'].astype(str).str.lower()
is_buy = dir_lower.str.contains('buy')
is_sell = dir_lower.str.contains('sell')

order_summary['favorable_spread_pct'] = np.where(
    is_buy,
    (order_summary['market_price_at_submit'] - order_summary['limit_price']) / order_summary['market_price_at_submit'],
    np.where(
        is_sell,
        (order_summary['limit_price'] - order_summary['market_price_at_submit']) / order_summary['market_price_at_submit'],
        np.nan,
    ),
)
order_summary['favorable_spread_bps'] = order_summary['favorable_spread_pct'] * 10000
order_summary['days_to_final'] = (order_summary['final_at'] - order_summary['submitted_at']).dt.days.fillna(0)

# Keep rows that have spread data
analysis_df = order_summary.dropna(subset=['favorable_spread_bps']).copy()
if analysis_df.empty:
    raise ValueError('No orders with market_price_at_submit found. Re-run backtest after logging update.')

print(f'orders with spread data: {len(analysis_df):,}')
display(analysis_df.head())

In [ ]:
# Bucket spread in bps for fill-rate curve
bins = [0, 25, 50, 75, 100, 125, 150, 200, 300, 500, np.inf]
labels = ['0-25', '25-50', '50-75', '75-100', '100-125', '125-150', '150-200', '200-300', '300-500', '500+']

analysis_df['spread_bucket_bps'] = pd.cut(
    analysis_df['favorable_spread_bps'].clip(lower=0), bins=bins, labels=labels, right=False
)

bucket_stats = (
    analysis_df.groupby('spread_bucket_bps', observed=False)
               .agg(
                   orders=('order_id', 'count'),
                   avg_fill_pct=('fill_pct', 'mean'),
                   full_fill_rate=('is_full_fill', 'mean')
               )
               .reset_index()
)

bucket_stats['avg_fill_pct'] = bucket_stats['avg_fill_pct'].fillna(0.0)
bucket_stats['full_fill_rate'] = bucket_stats['full_fill_rate'].fillna(0.0)

tier_stats = (
    analysis_df.groupby('tier', as_index=False)
               .agg(
                   orders=('order_id', 'count'),
                   avg_spread_bps=('favorable_spread_bps', 'mean'),
                   avg_fill_pct=('fill_pct', 'mean'),
                   full_fill_rate=('is_full_fill', 'mean')
               )
               .sort_values('orders', ascending=False)
)

fig, axes = plt.subplots(2, 2, figsize=(16, 11))

sns.histplot(analysis_df['favorable_spread_bps'], bins=25, ax=axes[0, 0], color='#1f77b4')
axes[0, 0].set_title('Spread at Submit (bps)')
axes[0, 0].set_xlabel('Favorable spread (bps)')
axes[0, 0].grid(alpha=0.3)

sns.scatterplot(
    data=analysis_df,
    x='favorable_spread_bps',
    y='fill_pct',
    hue='tier',
    alpha=0.6,
    ax=axes[0, 1]
)
axes[0, 1].set_title('Fill % vs Spread (per order)')
axes[0, 1].set_xlabel('Favorable spread (bps)')
axes[0, 1].set_ylabel('Fill %')
axes[0, 1].set_ylim(-0.02, 1.02)
axes[0, 1].grid(alpha=0.3)

axes[1, 0].plot(bucket_stats['spread_bucket_bps'].astype(str), bucket_stats['full_fill_rate'], marker='o', label='Full fill rate')
axes[1, 0].plot(bucket_stats['spread_bucket_bps'].astype(str), bucket_stats['avg_fill_pct'], marker='o', label='Avg fill %')
axes[1, 0].set_title('Fill Rates by Spread Bucket')
axes[1, 0].set_xlabel('Favorable spread bucket (bps)')
axes[1, 0].set_ylabel('Rate')
axes[1, 0].set_ylim(0, 1.02)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

sns.barplot(data=tier_stats, x='tier', y='avg_fill_pct', ax=axes[1, 1], color='#2ca02c')
axes[1, 1].set_title('Average Fill % by Tier')
axes[1, 1].set_xlabel('Tier')
axes[1, 1].set_ylabel('Average fill %')
axes[1, 1].set_ylim(0, 1.02)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print('Spread-bucket stats')
display(bucket_stats)

print('Tier stats')
display(tier_stats)